# README

### Inserting another Database:
    1) have the last column be Y and the previous columns be X
    
    2) Add database to name df_list_name and refernce to df_list
        -- after clean up section
        
    3) Add another df object to the stats class

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from sklearn import svm
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier  

# Load Data Files from UCI ML

In [2]:
#df_winequality_red = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv')
#abalone_header = ['sex', 'len', 'diam','height','tot-weight','shuck_weigth','Visc_weight;]
#df_tele = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data', names = tele_header)

df_winequality_red = pd.read_csv('winequality-red.csv')
df_winequality_whi = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')

abalone_header = ['sex', 'len', 'diam','height','tot-weight','shuck_weigth','Visc_weight', 'shell_weight', 'rings']
df_abalone =pd.read_csv('abalone.data', names = abalone_header)

tele_header = ['fLen', 'fWidth', 'fsize', 'fConc', 'fConc1', 'fAsym','fM3Long','fM3Trans','fAlpha','fDist','class']
df_tele = pd.read_csv('magic04.data', names = tele_header)

df_iris = pd.read_csv('Iris.csv')

#df_adult = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data')

# Make df_Tele 10% its normal size
df_tele = df_tele.sample(frac = 0.05)
df_abalone = df_abalone.sample(frac = 0.1)
df_winequality_whi = df_winequality_whi.sample(frac = 0.2)
df_winequality_red = df_winequality_whi.sample(frac = 0.3)

In [3]:
print(df_winequality_red.shape)
print(df_winequality_whi.shape)
print(df_abalone.shape)
print(df_tele.shape)

(294, 12)
(980, 12)
(418, 9)
(951, 11)


# Clean adult database

In [4]:
#pd.get_dummies(pd.Series(list(df_adult['Bachelors'].values)))

# Clean abalone df 

In [5]:
df_abalone.insert(0, 'infant', df_abalone['sex'] == 'I')
df_abalone['infant'] = df_abalone['infant'].astype(int)

df_abalone.insert(0, 'male', df_abalone['sex'] == 'M')
df_abalone['male'] = df_abalone['male'].astype(int)

df_abalone.insert(0, 'female', df_abalone['sex'] == 'F')
df_abalone['female'] = df_abalone['female'].astype(int)

df_abalone['old'] = df_abalone['rings'] >= df_abalone['rings'].mean()
df_abalone['old'] = df_abalone['old'].astype(int)

df_abalone = df_abalone.drop(columns = ['sex'])
df_abalone = df_abalone.drop(columns = ['rings'])

In [6]:
df_abalone.head()

,female,male,infant,len,diam,height,tot-weight,shuck_weigth,Visc_weight,shell_weight,old
3495,0,1,0,0.560,0.415,0.130,0.7615,0.3695,0.1700,0.1955,0
3549,0,0,1,0.500,0.365,0.125,0.5280,0.2290,0.1030,0.1645,0
657,1,0,0,0.615,0.515,0.170,1.1400,0.4305,0.2245,0.4200,1
3109,0,0,1,0.380,0.290,0.085,0.2285,0.0880,0.0465,0.0750,0
361,1,0,0,0.590,0.465,0.150,0.9970,0.3920,0.2460,0.3400,1


# Clean Wine Quality DF

In [7]:
df_winequality_whi['qaulity_y'] = df_winequality_whi['quality'] >= 6
df_winequality_whi['qaulity_y'] = df_winequality_whi['qaulity_y'].astype(int)
df_winequality_whi = df_winequality_whi.drop(columns = ['quality'])

df_winequality_red['qaulity_y'] = df_winequality_red['quality'] >= 6
df_winequality_red['qaulity_y'] = df_winequality_red['qaulity_y'].astype(int)
df_winequality_red = df_winequality_red.drop(columns = ['quality'])

In [8]:
df_winequality_red.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,qaulity_y
3469,7.6,0.22,0.28,12.00,0.056,68.0,143.0,0.99830,2.99,0.30,9.2,1
2481,6.4,0.37,0.37,4.85,0.041,39.5,216.5,0.99432,3.10,0.50,9.8,1
1448,7.4,0.16,0.49,1.20,0.055,18.0,150.0,0.99170,3.23,0.47,11.2,1
3685,6.8,0.15,0.41,12.90,0.044,79.5,183.0,0.99742,3.24,0.78,10.2,1
1022,6.8,0.19,0.32,7.05,0.019,54.0,188.0,0.99350,3.25,0.37,11.1,1


# Clean Tele df

In [9]:
df_tele['class'] = df_tele['class'] == 'g'
df_tele['class'] = df_tele['class'].astype(int)

In [10]:
df_tele.head()

,fLen,fWidth,fsize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
4367,50.9510,15.9968,2.9337,0.3588,0.2242,-49.0793,40.1485,-4.2133,4.0907,281.905,1
8391,28.5756,12.4137,2.5916,0.4507,0.2446,18.6866,20.6222,-6.2372,15.4800,212.354,1
10499,44.3695,16.8796,2.7271,0.2699,0.1396,57.5556,38.4910,-14.2284,0.5770,240.020,1
7488,89.4871,33.6833,3.9396,0.1254,0.0682,13.7476,74.5934,-29.4768,0.0090,280.334,1
1453,38.2290,16.5916,2.5308,0.3711,0.1900,-56.5835,20.3266,-10.0589,3.7101,101.977,1


# Clean Iris DF

In [11]:
# set df_iris['y'] == 1 if its setosa or vericolor
df_iris['y'] = (df_iris['Species'] == 'Iris-setosa') | (df_iris['Species'] == 'Iris-versicolor')
df_iris = df_iris.drop(columns = ['Id', 'Species'])
df_iris['y'] = df_iris['y'].astype(int)

In [12]:
df_iris.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,y
0,5.1,3.5,1.4,0.2,1
1,4.9,3.0,1.4,0.2,1
2,4.7,3.2,1.3,0.2,1
3,4.6,3.1,1.5,0.2,1
4,5.0,3.6,1.4,0.2,1


# Lists for grid Search

In [13]:
C_list    = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
depth_set = [1, 2, 3, 4, 5]
k_list    = [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 15, 17, 21, 25, 29, 31, 33]

df_name_list = ['Wine Quality Red', 'Abalone', 'Telescope', 'Wine Quality White']
df_list = [df_winequality_red, df_abalone, df_tele, df_winequality_whi]

In [14]:
def test_train_split(df, perc):    
    X_and_Y = df.values
    np.random.shuffle(X_and_Y)    # Shuffle the data.
    
    X = X_and_Y[:,:X_and_Y.shape[1]-2]  # Shape: (300,)
    Y = X_and_Y[:,X_and_Y.shape[1]-1].reshape((X_and_Y.shape[0],1)) 

    train_Num = int(X.shape[0]*perc)

    X_train_val = X[: train_Num, :] # Get features from train + val set.
    X_test      = X[train_Num :, :] # Get features from test set.     
    Y_train_val = Y[: train_Num, :] # Get labels from train + val set.
    Y_test      = Y[train_Num :, :] # Get labels from test set.  

    return (X_train_val, X_test, Y_train_val, Y_test)    

# Grid Search Functions

In [15]:
def svc_grid(X_train_val, Y_train_val):    
    classifier = svm.SVC(kernel= 'linear', C=C_list)
    parameters = {'C':C_list}

    grid_Search = GridSearchCV(classifier, parameters, cv=3, return_train_score='true', scoring='accuracy')
    grid_Search.fit(X_train_val, Y_train_val.reshape(-1))
    
    return grid_Search

def dec_tree_grid(X_train_val, Y_train_val):    
    classifier = tree.DecisionTreeClassifier()
    params = {'criterion': ['entropy'], 'max_depth' : depth_set}

    grid_Search = GridSearchCV(classifier, params, cv=3, return_train_score = 'true', scoring='accuracy')
    grid_Search.fit(X_train_val, Y_train_val.reshape(-1))
  
    return grid_Search

def knn_grid(X_train_val, Y_train_val):       
    classifier = KNeighborsClassifier()    
    
    param_grid = dict(n_neighbors=k_list)
   
    grid_Search = GridSearchCV(classifier, param_grid, cv=3, return_train_score = 'true', scoring='accuracy')
    grid_Search.fit(X_train_val, Y_train_val.reshape(-1))
  
    return grid_Search

# Draw heatmaps for data

In [16]:
def compile_heatmaps(grid_Search, params):
    train_acc = np.array([grid_Search.cv_results_['mean_train_score']]).reshape(len(params),1)
    draw_heatmap_linear(train_acc, 'train accuracy', params)

    val_acc = np.array([grid_Search.cv_results_['mean_test_score']]).reshape(len(params),1)
    draw_heatmap_linear(val_acc, 'val accuracy', params)

def draw_heatmap_linear(acc, acc_desc, params):
    plt.figure(figsize = (2,4))
    ax = sns.heatmap(acc, annot=True, fmt='.3f', yticklabels=params, xticklabels=[])
    ax.collections[0].colorbar.set_label("accuracy")
    ax.set(ylabel='$C$')
    plt.title(acc_desc + ' w.r.t $C$')
    sns.set_style("whitegrid", {'axes.grid' : False})
    plt.show()

# 3 Different Algorithms

In [17]:
def fit_best_est(grid_Search, X_train_val, X_test, Y_train_val, Y_test):
    
        clf = grid_Search.best_estimator_
        clf.fit(X_train_val, Y_train_val.reshape(-1))

        test_acc  = clf.score(X_test, Y_test.reshape(-1))
        train_acc = grid_Search.cv_results_['mean_train_score'][grid_Search.best_index_]
        val_acc   = grid_Search.cv_results_['mean_test_score'][grid_Search.best_index_]

        if show_heatmaps:
            compile_heatmaps(grid_Search, depth_set)
            print("Best parameters: ", grid_Search.best_params_)   
            
        return test_acc, train_acc, val_acc

In [18]:
def do_svc(df, perc):
    X_train_val, X_test, Y_train_val, Y_test = test_train_split(df, perc)
    grid_Search = svc_grid(X_train_val, Y_train_val)  
    
    return fit_best_est(grid_Search, X_train_val, X_test, Y_train_val, Y_test)

def do_dec_tree(df, perc):
    X_train_val, X_test, Y_train_val, Y_test = test_train_split(df, perc)
    grid_Search = dec_tree_grid(X_train_val, Y_train_val)     
    
    return fit_best_est(grid_Search, X_train_val, X_test, Y_train_val, Y_test)

def do_knn(df, perc):
    X_train_val, X_test, Y_train_val, Y_test = test_train_split(df, perc)
    grid_Search = knn_grid(X_train_val, Y_train_val)  
     
    return fit_best_est(grid_Search, X_train_val, X_test, Y_train_val, Y_test)

# Data Structure to Hold all of the accuracies

In [19]:
num_kernels = 3
train_list = [0.2, 0.5, 0.8]

class stats_i:
    def __init__(self):
        self.acc = {
          "train_acc": [],
          "val_acc"  : [],
          "test_acc" : []
        }
    def populate_stats(self, test_acc, train_acc, val_acc):
        self.acc['train_acc'].append(int(train_acc*1000)/1000)
        self.acc['test_acc'].append(int(test_acc*1000)/1000)
        self.acc['val_acc'].append(int(val_acc*1000)/1000)       
        
class stats: 
    def __init__(self):   
        self.df1 = stats_i()
        self.df2 = stats_i()
        self.df3 = stats_i()
        self.df4 = stats_i()
        self.df = [self.df1, self.df2, self.df3, self.df4]
        
    def print(self):
        print('---------------| Statistics |-----------------')
    
        itr = 0
        for df in self.df:
            print('DataFrame %d: ' %(itr+1), df_name_list[itr])
            print('                        0.2%   0.5%   0.8%')
            print('Training Accuracy:   ', df.acc['train_acc'])
            print('Testing Accuracy:    ', df.acc['test_acc'])
            print('Validation Accuracy: ', df.acc['val_acc'])
            print()
            itr += 1

In [20]:
show_heatmaps = False

In [21]:
import warnings
warnings.filterwarnings("ignore")

csv_stats = stats()
dec_tree_stats = stats()
knn_stats = stats()

num_percentage_trials = 3

# loop over percentages
for perc in train_list:
    
    # loop over each dataset in list
    for i in range(len(df_list)):
        test_acc_sum, val_acc_sum, train_acc_sum = 0,0,0
        for x in range(num_percentage_trials):
            test_acc, train_acc, val_acc = do_svc(df_list[i], perc)
            test_acc_sum += test_acc
            train_acc_sum += train_acc
            val_acc_sum += val_acc
        csv_stats.df[i].populate_stats(test_acc_sum/num_percentage_trials, 
                                       val_acc_sum/num_percentage_trials, 
                                       train_acc_sum/num_percentage_trials)

# loop over each dataset in list
    for i in range(len(df_list)):
        test_acc_sum, val_acc_sum, train_acc_sum = 0,0,0
        for x in range(num_percentage_trials):
            test_acc, train_acc, val_acc = do_dec_tree(df_list[i], perc)
            test_acc_sum += test_acc
            train_acc_sum += train_acc
            val_acc_sum += val_acc
        dec_tree_stats.df[i].populate_stats(test_acc_sum/num_percentage_trials, 
                                       val_acc_sum/num_percentage_trials, 
                                       train_acc_sum/num_percentage_trials)

# loop over each dataset in list
    for i in range(len(df_list)):
        test_acc_sum, val_acc_sum, train_acc_sum = 0,0,0
        for x in range(num_percentage_trials):
            test_acc, train_acc, val_acc = do_knn(df_list[i], perc)
            test_acc_sum += test_acc
            train_acc_sum += train_acc
            val_acc_sum += val_acc
        knn_stats.df[i].populate_stats(test_acc_sum/num_percentage_trials, 
                                       val_acc_sum/num_percentage_trials, 
                                       train_acc_sum/num_percentage_trials)

In [22]:
print('---------------|    SVM     |-----------------')
csv_stats.print()

---------------|    SVM     |-----------------
---------------| Statistics |-----------------
DataFrame 1:  Wine Quality Red
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.741, 0.723, 0.717]
Testing Accuracy:     [0.707, 0.707, 0.7]
Validation Accuracy:  [0.752, 0.723, 0.717]

DataFrame 2:  Abalone
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.734, 0.728, 0.75]
Testing Accuracy:     [0.738, 0.736, 0.734]
Validation Accuracy:  [0.76, 0.745, 0.751]

DataFrame 3:  Telescope
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.775, 0.799, 0.799]
Testing Accuracy:     [0.784, 0.78, 0.752]
Validation Accuracy:  [0.787, 0.798, 0.8]

DataFrame 4:  Wine Quality White
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.685, 0.661, 0.667]
Testing Accuracy:     [0.658, 0.664, 0.646]
Validation Accuracy:  [0.683, 0.661, 0.667]



In [23]:
print('---------------|  Dec Tree  |-----------------')
print(dec_tree_stats.print())

---------------|  Dec Tree  |-----------------
---------------| Statistics |-----------------
DataFrame 1:  Wine Quality Red
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.712, 0.734, 0.739]
Testing Accuracy:     [0.68, 0.698, 0.672]
Validation Accuracy:  [0.853, 0.782, 0.797]

DataFrame 2:  Abalone
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.783, 0.751, 0.735]
Testing Accuracy:     [0.696, 0.732, 0.73]
Validation Accuracy:  [0.897, 0.807, 0.811]

DataFrame 3:  Telescope
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.775, 0.776, 0.784]
Testing Accuracy:     [0.749, 0.775, 0.781]
Validation Accuracy:  [0.818, 0.857, 0.849]

DataFrame 4:  Wine Quality White
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.693, 0.712, 0.723]
Testing Accuracy:     [0.675, 0.702, 0.688]
Validation Accuracy:  [0.844, 0.817, 0.785]

None


In [24]:
print('---------------|    KNN     |-----------------')
print(knn_stats.print())

---------------|    KNN     |-----------------
---------------| Statistics |-----------------
DataFrame 1:  Wine Quality Red
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.764, 0.732, 0.741]
Testing Accuracy:     [0.683, 0.707, 0.666]
Validation Accuracy:  [0.859, 0.73, 0.736]

DataFrame 2:  Abalone
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.775, 0.759, 0.776]
Testing Accuracy:     [0.706, 0.765, 0.753]
Validation Accuracy:  [0.74, 0.806, 0.792]

DataFrame 3:  Telescope
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.768, 0.741, 0.766]
Testing Accuracy:     [0.729, 0.759, 0.757]
Validation Accuracy:  [0.857, 0.797, 0.811]

DataFrame 4:  Wine Quality White
                        0.2%   0.5%   0.8%
Training Accuracy:    [0.692, 0.691, 0.677]
Testing Accuracy:     [0.65, 0.644, 0.637]
Validation Accuracy:  [0.822, 0.82, 0.75]

None
